In [1]:
import pandas as pd
import numpy as np
import os
os.chdir(r'D:\lym本科\科研相关\华为项目资料\2025-1-23\Encoder_PELT')   # set as your own working directory

In [2]:
data_path = r'D:\lym本科\科研相关\华为项目资料\2025-1-23\Encoder_PELT\data\SMD\Server-Machine-Dataset-main\Server-Machine-Dataset-main\processed_csv'   # set as your own working directory

for machine_folder in sorted(os.listdir(data_path)):
    if machine_folder.startswith('machine-1-'):
        machine_path = os.path.join(data_path, machine_folder)

        for file_type in ['train', 'test']:
            csv_file = os.path.join(machine_path, f'{machine_folder}_{file_type}.csv')
            
            if os.path.exists(csv_file):
                try:
                    df = pd.read_csv(csv_file)

                    print(f"\n=== {machine_folder}/{machine_folder}_{file_type}.csv ===")
                    print(f"Path: {csv_file}")
                    print(f"Number of samples (rows): {len(df)}")
                    print(f"Number of features (columns): {len(df.columns)}")
                    print("First 5 rows of data:")
                    print(df.head())
                    print("Column names:", df.columns.tolist())
                    print("Data types:")
                    print(df.dtypes)
                    
                except Exception as e:
                    print(f"Error processing file {csv_file}: {str(e)}")
            else:
                print(f"File not found: {csv_file}")

print("\nAll files processed.")


=== machine-1-1/machine-1-1_train.csv ===
Path: D:\lym本科\科研相关\华为项目资料\2025-1-23\Encoder_PELT\data\SMD\Server-Machine-Dataset-main\Server-Machine-Dataset-main\processed_csv\machine-1-1\machine-1-1_train.csv
Number of samples (rows): 28479
Number of features (columns): 39
First 5 rows of data:
   timestamp     col_0     col_1     col_2     col_3  col_4     col_5  \
0          0  0.032258  0.039195  0.027871  0.024390    0.0  0.915385   
1          1  0.043011  0.048729  0.033445  0.025552    0.0  0.915385   
2          2  0.043011  0.034958  0.032330  0.025552    0.0  0.915385   
3          3  0.032258  0.028602  0.030100  0.024390    0.0  0.912821   
4          4  0.032258  0.019068  0.026756  0.023229    0.0  0.912821   

      col_6  col_7     col_8  ...  col_28    col_29    col_30    col_31  \
0  0.343691    0.0  0.020011  ...     0.0  0.004298  0.029993  0.022131   
1  0.344633    0.0  0.019160  ...     0.0  0.004298  0.030041  0.028821   
2  0.344633    0.0  0.020011  ...     0.0  

In [3]:
def create_windows(data, window_size):
    windows = []
    for i in range(len(data) - window_size + 1):
        window = data[i:i+window_size,:]
        windows.append(window)
    return np.array(windows)

In [4]:
base_path = r'D:\lym本科\科研相关\华为项目资料\2025-1-23\Encoder_PELT\data\SMD\Server-Machine-Dataset-main\Server-Machine-Dataset-main\processed_csv'   # set as your own working directory

train_path = os.path.join(base_path, 'machine-1-1', 'machine-1-1_train.csv')
train_df = pd.read_csv(train_path)
train_data = train_df.drop(columns=['timestamp']).values
print(f"Training data shape: {train_data.shape}")

test_data_list = []
test_label_list = []

for i in range(1, 12):
    test_path = os.path.join(base_path, f'machine-3-{i}', f'machine-3-{i}_test.csv')
    if os.path.exists(test_path):
        test_df = pd.read_csv(test_path)
        test_data = test_df.drop(columns=['timestamp', 'label']).values
        test_labels = test_df['label'].values
        test_data_list.append(test_data)
        test_label_list.append(test_labels)
        print(f"machine-3-{i} test data shape: {test_data.shape}, label shape: {test_labels.shape}")
    else:
        print(f"Warning: {test_path} does not exist")

combined_test_data = np.concatenate(test_data_list, axis=0)
combined_test_labels = np.concatenate(test_label_list, axis=0)
print(f"Combined test data shape: {combined_test_data.shape}, label shape: {combined_test_labels.shape}")

window_size = 10

train_windows = create_windows(train_data, window_size)
train_windows = train_windows.reshape(train_windows.shape[0],train_windows.shape[2]*window_size)
print(f"Training window data shape: {train_windows.shape}")

test_windows = create_windows(combined_test_data, window_size)
test_windows = test_windows.reshape(test_windows.shape[0],test_windows.shape[2]*window_size)
test_windows_labels = combined_test_labels[window_size-1:]
print(f"Test window data shape: {test_windows.shape}, test window label shape: {test_windows_labels.shape}")

print("\nTest label distribution:")
unique, counts = np.unique(test_windows_labels, return_counts=True)
print(dict(zip(unique, counts)))

np.save(f'./data/SMD/train_{window_size}_v3.npy', train_windows)
np.save(f'./data/SMD/test_{window_size}_v3.npy', test_windows)
np.save(f'./data/SMD/test_label_{window_size}_v3.npy', test_windows_labels)

Training data shape: (28479, 38)
machine-3-1 test data shape: (28700, 38), label shape: (28700,)
machine-3-2 test data shape: (23703, 38), label shape: (23703,)
machine-3-3 test data shape: (23703, 38), label shape: (23703,)
machine-3-4 test data shape: (23687, 38), label shape: (23687,)
machine-3-5 test data shape: (23691, 38), label shape: (23691,)
machine-3-6 test data shape: (28726, 38), label shape: (28726,)
machine-3-7 test data shape: (28705, 38), label shape: (28705,)
machine-3-8 test data shape: (28704, 38), label shape: (28704,)
machine-3-9 test data shape: (28713, 38), label shape: (28713,)
machine-3-10 test data shape: (23693, 38), label shape: (23693,)
machine-3-11 test data shape: (28696, 38), label shape: (28696,)
Combined test data shape: (290721, 38), label shape: (290721,)
Training window data shape: (28470, 380)
Test window data shape: (290712, 380), test window label shape: (290712,)

Test label distribution:
{0.0: 282713, 1.0: 7999}
